In [ ]:
import pandas as pd
from mosqlient import get_infodengue
from geodata.features import InfodengueMaps
import geopandas as gpd
import plotly.express as px

In [ ]:
# Initialize Infodengue and get Brasil map
casos = get_infodengue(
    start_date='2022-12-30',
    end_date='2023-01-30',
    disease='dengue',
    uf='RJ'
)
casos

In [ ]:
casos.info()

In [ ]:
# casos.set_index('municipio_nome', inplace=True)
casos = casos.groupby(['municipio_geocodigo', 'municipio_nome']).sum()
casos

In [ ]:
IM = InfodengueMaps()
map_geojson = IM.get_state_geojson("RJ")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(map_geojson)
gdf['CD_MUN'] = gdf['CD_MUN'].astype(int)
gdf.plot()

In [ ]:
rio = IM.get_city_geojson(city_code='3304557')
rio
# gpd.GeoDataFrame.from_features(rio).plot()

In [ ]:
mapa = pd.merge(gdf, casos.reset_index(), left_on='CD_MUN', right_on='municipio_geocodigo', how='left')
ax= mapa.plot(column='casos', scheme='natural_breaks',legend=True, legend_kwds={'bbox_to_anchor': (1.32, 1)})
ax.set_axis_off()

In [ ]:
fig = px.choropleth(mapa, geojson=mapa.geometry, color='casos', locations=mapa.index, hover_name='municipio_nome')
fig.update_geos(fitbounds="locations", visible=False)

In [ ]:
gdf

In [ ]:
gdf[['NM_MUN', 'CD_MUN']].drop_duplicates().to_dict(orient='records')